https://www.machinelearningplus.com/nlp/gensim-tutorial/

https://github.com/marcmuon/nlp_yelp_review_unsupervised/blob/master/notebooks/2-train_corpus_prep_and_LDA_train.ipynb

https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28

## 从句子中构建字典

In [4]:
import gensim
from gensim import corpora
from pprint import pprint
# 语料
documents = ["The Saudis are preparing a report that will acknowledge that", 
             "Saudi journalist Jamal Khashoggi's death was the result of an", 
             "interrogation that went wrong, one that was intended to lead", 
             "to his abduction from Turkey, according to two sources."]

documents_2 = ["One source says the report will likely conclude that", 
                "the operation was carried out without clearance and", 
                "transparency and that those involved will be held", 
                "responsible. One of the sources acknowledged that the", 
                "report is still being prepared and cautioned that", 
                "things could change."]

# 分词
texts=[[text for text in doc.split()] for doc in documents]
# 创建字典
dictionary = corpora.Dictionary(texts)
# 打印字典的信息
print(dictionary)

Dictionary(33 unique tokens: ['Saudis', 'The', 'a', 'acknowledge', 'are']...)


In [5]:
# 词语到id的映射
print(dictionary.token2id)

{'Saudis': 0, 'The': 1, 'a': 2, 'acknowledge': 3, 'are': 4, 'preparing': 5, 'report': 6, 'that': 7, 'will': 8, 'Jamal': 9, "Khashoggi's": 10, 'Saudi': 11, 'an': 12, 'death': 13, 'journalist': 14, 'of': 15, 'result': 16, 'the': 17, 'was': 18, 'intended': 19, 'interrogation': 20, 'lead': 21, 'one': 22, 'to': 23, 'went': 24, 'wrong,': 25, 'Turkey,': 26, 'abduction': 27, 'according': 28, 'from': 29, 'his': 30, 'sources.': 31, 'two': 32}


In [7]:
# 添加新的文档并更新词典
texts_2=[[text for text in doc.split()] for doc in documents_2]
dictionary.add_documents(texts_2)
print(dictionary)
print(dictionary.token2id)

Dictionary(60 unique tokens: ['Saudis', 'The', 'a', 'acknowledge', 'are']...)
{'Saudis': 0, 'The': 1, 'a': 2, 'acknowledge': 3, 'are': 4, 'preparing': 5, 'report': 6, 'that': 7, 'will': 8, 'Jamal': 9, "Khashoggi's": 10, 'Saudi': 11, 'an': 12, 'death': 13, 'journalist': 14, 'of': 15, 'result': 16, 'the': 17, 'was': 18, 'intended': 19, 'interrogation': 20, 'lead': 21, 'one': 22, 'to': 23, 'went': 24, 'wrong,': 25, 'Turkey,': 26, 'abduction': 27, 'according': 28, 'from': 29, 'his': 30, 'sources.': 31, 'two': 32, 'One': 33, 'conclude': 34, 'likely': 35, 'says': 36, 'source': 37, 'and': 38, 'carried': 39, 'clearance': 40, 'operation': 41, 'out': 42, 'without': 43, 'be': 44, 'held': 45, 'involved': 46, 'those': 47, 'transparency': 48, 'acknowledged': 49, 'responsible.': 50, 'sources': 51, 'being': 52, 'cautioned': 53, 'is': 54, 'prepared': 55, 'still': 56, 'change.': 57, 'could': 58, 'things': 59}


## 创建词袋

In [8]:
from gensim.utils import simple_preprocess
my_docs = ["Who let the dogs out?",
           "Who? Who? Who? Who?"]

# 分词
tokenized_list = [simple_preprocess(doc) for doc in my_docs]

mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]
pprint(mycorpus)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(4, 4)]]


In [10]:
word_counts = [[(mydict[id], count) for id, count in line] for line in mycorpus]
pprint(word_counts)

[[('dogs', 1), ('let', 1), ('out', 1), ('the', 1), ('who', 1)], [('who', 4)]]


('dogs', 1)代表dogs在第一句话中出现了一次

## 创建Tfidf矩阵

In [17]:
from gensim import models
import numpy as np

documents = ["This is the first line",
             "This is the second sentence",
             "This third document"]

# 创建词典和语料
mydict = corpora.Dictionary([simple_preprocess(line) for line in documents])
corpus = [mydict.doc2bow(simple_preprocess(line)) for line in documents]

# 打印每个doc的词频
for doc in corpus:
    print([[mydict[id], freq] for id, freq in doc])

[['first', 1], ['is', 1], ['line', 1], ['the', 1], ['this', 1]]
[['is', 1], ['the', 1], ['this', 1], ['second', 1], ['sentence', 1]]
[['this', 1], ['document', 1], ['third', 1]]


In [20]:
# 创建Tfidf模型
tfidf = models.TfidfModel(corpus, smartirs='ntc')

# 打印Tfidf权重
for doc in tfidf[corpus]:
    print([[mydict[id], np.around(freq, decimals=2)] for id, freq in doc])

[['first', 0.63], ['is', 0.31], ['line', 0.63], ['the', 0.31], ['this', 0.13]]
[['is', 0.31], ['the', 0.31], ['this', 0.13], ['second', 0.63], ['sentence', 0.63]]
[['this', 0.15], ['document', 0.7], ['third', 0.7]]


## 创建LDA模型

In [29]:
# step0：导入库和停用词
import gensim.downloader as api
from gensim.models import LdaModel, LdaMulticore
import gensim.downloader as api
from gensim.utils import simple_preprocess, lemmatize
from nltk.corpus import stopwords
import re
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
stop_words = stopwords.words('english')
stop_words = stop_words + ['com', 'edu', 'subject', 'lines', 'organization', 'would', 'article', 'could']

In [30]:
# step1：导入text8数据集
dataset = api.load("text8")
data = [d for d in dataset]

In [31]:
# step2 预处理数据
data_processed = []

for i, doc in enumerate(data[:100]):
    doc_out = []
    for wd in doc:
        if wd not in stop_words:  # remove stopwords
            lemmatized_word = lemmatize(wd, allowed_tags=re.compile('(NN|JJ|RB)'))  # lemmatize
            if lemmatized_word:
                doc_out = doc_out + [lemmatized_word[0].split(b'/')[0].decode('utf-8')]
        else:
            continue
    data_processed.append(doc_out)
print(data_processed[0][:5]) 


BadZipFile: File is not a zip file

https://github.com/EricSchles/sklearn_gensim_example/blob/master/example.py